<a href="https://colab.research.google.com/github/hhauptman/CS5350/blob/main/ID3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Method

In [63]:
import math
from collections import deque

ENTROPY = 0
MAJORITY_ERROR = 1
GINI_INDEX = 2

class Node:
    def __init__(self):
        # feature being examined
        self.value = None
        
        self.next = None
        # children (branches that represent attribute values)
        self.children = None

# sets up necessary information to run the algorithm
class Classifier:
    def __init__(self, S, featureNames, labels, uniqueLabels, method=ENTROPY):
        self.root = None

        self.method = method
        self.S = S
        self.featureNames = featureNames
        # outcomes
        self.labels = labels
        self.uniqueLabels = uniqueLabels

        # array that counts how many labels are in each category
        self.labelCategoriesCount = [list(labels).count(i) for i in self.uniqueLabels]
        # calculates initial entropy
        self.entropy = self.getEntropy([i for i in range(len(self.labels))])

    def getEntropy(self, sIndexes):
        if self.method == ENTROPY :
          return self.getEntropyStandard(sIndexes)
        elif self.method == MAJORITY_ERROR:
          return self.getMajorityError(sIndexes)
        elif self.method == GINI_INDEX:
          return self.getGiniIndex(sIndexes)
        else:
          print("yoU bROkE iT.") # should be unreachable :)
          return entropy

# METHODS FOR CALCULATING ENTROPY/PURITY #
    def getEntropyStandard(self, sIndexes):
        # sorted labels by index
        labels = [self.labels[i] for i in sIndexes]
        # count number of instances in each unique label
        lblCount = [labels.count(i) for i in self.uniqueLabels]
        return sum([-count / len(sIndexes) * math.log(count / len(sIndexes), 2) if count else 0 for count in lblCount])
  
    def getMajorityError(self, sIndexes):
        # sorted labels by index
        labels = [self.labels[i] for i in sIndexes]
        # count number of instances in each unique label
        lblCount = [labels.count(i) for i in self.uniqueLabels]
        majorityLabel = lblCount.index(max(lblCount))
        return (len(sIndexes)-lblCount[majorityLabel])/len(sIndexes)
  
    def getGiniIndex(self, sIndexes):
        # sorted labels by index
        labels = [self.labels[i] for i in sIndexes]
        # count number of instances in each unique label
        lblCount = [labels.count(i) for i in self.uniqueLabels]
        return 1 - sum([(count/len(sIndexes))**2 if count else 0 for count in lblCount])
# END METHODS #############################

    # calculate information gain using the original method discussed in lecture (expected reduction in entropy)
    def calcGain(self, sIndexes, fid):
        initEntropy = self.getEntropy(sIndexes)

        sFeatures = [self.S[i][fid] for i in sIndexes]
        uniqueVals = list(set(sFeatures))
        valCounts = [sFeatures.count(i) for i in uniqueVals]

        uniqueIndexes = [
            [sIndexes[i]
            for i, x in enumerate(sFeatures)
            if x == y]
            for y in uniqueVals
        ]
        
        newEntropy = sum([valCounts / len(sIndexes) * self.getEntropy(uniqueIndexes) for valCounts, uniqueIndexes in zip(valCounts, uniqueIndexes)])
        return initEntropy - newEntropy

    # find the attribute that maximizes information gain
    def calcMaxGain(self, sIndexes, featureIndexes):
        # get the entropy for each feature
        featureEntropies = [self.calcGain(sIndexes, fid) for fid in featureIndexes]
        maxGainIndex = featureIndexes[featureEntropies.index(max(featureEntropies))]

        # print("Splitting "," on ", self.featureNames[maxGainIndex],".")
        return self.featureNames[maxGainIndex], maxGainIndex

    # overhead for the recursive call to ID3 algorithm
    def ID3(self):
        sIndexes = [i for i in range(len(self.S))]
        featureIndexes = [i for i in range(len(self.featureNames))]
        self.root = self.ID3Recurse(sIndexes, featureIndexes, self.root)

    # recursive id3 function
    def ID3Recurse(self, sIndexes, featureIndexes, node):
        if not node:
            node = Node() 

        nodeLabels = [self.labels[i] for i in sIndexes]
        if len(set(nodeLabels)) == 1: # all labels the same (pure)
             node.value = self.labels[sIndexes[0]]
             # print("Node: ", node.value)
             return node

        if len(featureIndexes) == 0:  # no remaining features
            node.value = max(set(nodeLabels), key=nodeLabels.count)  # return 'most probable' leaf
            # print("Node: ", node.value)
            return node

        # otherwise...

        # choose the feature that maximizes the information gain
        maxName, maxIndex = self.calcMaxGain(sIndexes, featureIndexes)
        node.value = maxName
        node.children = []
        # values of the "best" feature
        maxFeatureVals = list(set([self.S[i][maxIndex] for i in sIndexes]))
        # print("Node (split): ", node.value)

        for val in maxFeatureVals:
            child = Node()

            # add branch (feature)
            child.value = val  
            node.children.append(child) 

            childIndexes = [i for i in sIndexes if self.S[i][maxIndex] == val]
            
            if not childIndexes:  
                child.next = max(set(nodeLabels), key=nodeLabels.count) # the next child is set to the node with the most labels
            else:
                if featureIndexes and maxIndex in featureIndexes:
                    featureIndexes.pop(featureIndexes.index(maxIndex)) 

                # recursively call the algorithm
                child.next = self.ID3Recurse(childIndexes, featureIndexes, child.next)

        return node

    def printTree(self):
        print("Tree method =", self.method)
        if not self.root:
            print("ERROR: No tree")
            return
        nodes = deque()
        nodes.append(self.root)
        while len(nodes) > 0:
            node = nodes.popleft()
            print(node.value)
            if node.children:
                for child in node.children:
                    print('(', child.value, ')')
                    nodes.append(child.next)
            elif node.next:
                print("Next node: ", node.next)
        print("\n\n")

## Car Example

In [67]:
import numpy as np
import pandas as pd
import csv
from collections import deque

featureNames = ['buying','maint','doors','persons','lug_boot','safety']
uniqueLabels = ['unacc', 'acc', 'good', 'vgood']

data = []
labels = []

with open('HW1Data/car/train.csv', mode='r') as file:
    csvReader = csv.reader(file)
    count = 0
    for row in csvReader:
        count += 1
        #separate targets from predictors
        data.append(row[:6])
        labels.append(row[6])
    print(f'Processed {count} lines.')

# instantiate DecisionTreeClassifier
ID3ContextStandard = Classifier(S=data, featureNames=featureNames, labels=labels, uniqueLabels=uniqueLabels)
ID3ContextME = Classifier(S=data, featureNames=featureNames, labels=labels, uniqueLabels=uniqueLabels, method=MAJORITY_ERROR)
ID3ContextGI = Classifier(S=data, featureNames=featureNames, labels=labels, uniqueLabels=uniqueLabels, method=GINI_INDEX)

print("System entropy {:.4f}".format(ID3ContextStandard.entropy))
print("System entropy {:.4f}".format(ID3ContextME.entropy))
print("System entropy {:.4f}".format(ID3ContextGI.entropy))
print("\n")

# run algorithm to build my tree
ID3ContextStandard.ID3()
ID3ContextStandard.printTree()

ID3ContextME.ID3()
ID3ContextME.printTree()

ID3ContextGI.ID3()
ID3ContextGI.printTree()


Processed 1000 lines.
System entropy 1.2147
System entropy 0.3020
System entropy 0.4603


Tree method = 0
safety
( low )
( high )
( med )
unacc
persons
( more )
( 2 )
( 4 )
unacc
buying
( low )
( vhigh )
( high )
( med )
unacc
acc
maint
( low )
( vhigh )
( high )
( med )
unacc
acc
acc
lug_boot
( big )
( small )
acc
acc
vgood
vgood
doors
( 2 )
( 5more )
unacc
good



Tree method = 1
buying
( low )
( vhigh )
( high )
( med )
safety
( low )
( high )
( med )
unacc
unacc
unacc
unacc
persons
( more )
( 2 )
( 4 )
acc
maint
( low )
( vhigh )
( high )
( med )
unacc
vgood
lug_boot
( big )
( small )
acc
acc
vgood
vgood
doors
( 2 )
( 5more )
unacc
good



Tree method = 2
safety
( low )
( high )
( med )
unacc
persons
( more )
( 2 )
( 4 )
unacc
buying
( low )
( vhigh )
( high )
( med )
unacc
acc
maint
( low )
( vhigh )
( high )
( med )
unacc
acc
acc
lug_boot
( big )
( small )
acc
acc
vgood
vgood
doors
( 2 )
( 5more )
unacc
good





## Bank Example

In [68]:
import numpy as np
import pandas as pd
import csv
from collections import deque

featureNames = ['age','job','marital','education','default','balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']
uniqueLabels = ['yes', 'no']

data = []
labels = []

with open('HW1Data/bank/train.csv', mode='r') as file:
    csvReader = csv.reader(file)
    count = 0
    for row in csvReader:
        count += 1
        #separate targets from predictors
        data.append(row[:16])
        labels.append(row[16])
    print(f'Processed {count} lines.')

# instantiate DecisionTreeClassifier
ID3ContextStandard = Classifier(S=data, featureNames=featureNames, labels=labels, uniqueLabels=uniqueLabels)
ID3ContextME = Classifier(S=data, featureNames=featureNames, labels=labels, uniqueLabels=uniqueLabels, method=MAJORITY_ERROR)
ID3ContextGI = Classifier(S=data, featureNames=featureNames, labels=labels, uniqueLabels=uniqueLabels, method=GINI_INDEX)

print("System entropy {:.4f}".format(ID3ContextStandard.entropy))
print("System entropy {:.4f}".format(ID3ContextME.entropy))
print("System entropy {:.4f}".format(ID3ContextGI.entropy))
print("\n")

# run algorithm to build my tree
ID3ContextStandard.ID3()
ID3ContextStandard.printTree()

ID3ContextME.ID3()
ID3ContextME.printTree()

ID3ContextGI.ID3()
ID3ContextGI.printTree()

Streaming output truncated to the last 5000 lines.
( 2708 )
( -382 )
( 84 )
( 1957 )
( 7 )
( 2812 )
( -398 )
( 5674 )
( 673 )
( 4401 )
( 1642 )
( 3036 )
( 665 )
( 2142 )
( 2838 )
( 505 )
( 1975 )
( 710 )
( 2440 )
( 2610 )
( 1309 )
( 660 )
( 350 )
( 2236 )
( 997 )
( -43 )
( 5397 )
( 458 )
( 9531 )
( 5871 )
( 175 )
( 65 )
( 943 )
( 2000 )
( 4039 )
( 200 )
( 2876 )
( 9645 )
( 2271 )
( 1922 )
( 4269 )
( 1193 )
( 2378 )
( -417 )
( 3573 )
( 2974 )
( -249 )
( 984 )
( 16873 )
( -145 )
( 5270 )
( 2340 )
( 11512 )
( 1653 )
( 3587 )
( 1836 )
( 415 )
( 25 )
( 2389 )
( 862 )
( 367 )
( 636 )
( 1820 )
( 377 )
( 168 )
( -216 )
( 226 )
( 554 )
( 298 )
( 14692 )
( 506 )
( 162 )
( 9347 )
( 762 )
( 903 )
( -301 )
( 3876 )
( 1336 )
( 1708 )
( 87 )
( 1696 )
( -202 )
( 3340 )
( 2002 )
( 1811 )
( 4816 )
( 1762 )
( 4260 )
( 728 )
( 1121 )
( 342 )
( 1306 )
( 371 )
( 3398 )
( 2629 )
( 13360 )
( 3867 )
( -26 )
( -10 )
( 837 )
( 985 )
( 40 )
( 110 )
( 7742 )
( 1419 )
( 1815 )
( 3374 )
( 5909 )
( -499 )
( 402 )
( -